In [1]:
import os, sys
# for accessing src, stan, etc.
sys.path.append(os.path.abspath(os.path.join("../..")))

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

from src.models.HmmEam import model
from helpers import *

/Users/simonkucharsky/projects/bayesflow/amortized-mixture/.venv/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 8)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 400, 3)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


In [2]:
colors = {
    "Prior": '#999999',
    "BayesFlow": '#005b96',
    "Stan": '#7C0000'
}

In [3]:
subjects = sorted(os.listdir(os.path.join('dutilh-resources', 'data')))
subjects = [os.path.splitext(s)[0] for s in subjects]

prior = model.prior(4000)['prior_draws']

In [4]:
for subject in subjects:
    rts, choices = get_data(subject)

    path = os.path.join('dutilh-resources', 'bayesflow-fits', subject) + '.pkl'
    with open(path, 'rb') as f:
        bf_posterior, (bf_forward, bf_backward, bf_smoothing ) = pickle.load(f)
        bf_posterior = bf_posterior[0]
        bf_forward = bf_forward[0]
        bf_backward = bf_backward[0]
        bf_smoothing = bf_smoothing[0]


    path = os.path.join('dutilh-resources', 'stan-fits', subject) + '.pkl'
    with open(path, 'rb') as f:
        stan_fit = pickle.load(f)

    stan_posterior = np.array([
        stan_fit.stan_variable('transition_matrix')[:, 0, 0],
        stan_fit.stan_variable('transition_matrix')[:, 1, 1],
        stan_fit.stan_variable('alpha_1'),
        stan_fit.stan_variable('alpha_2'),
        stan_fit.stan_variable('nu_1'),
        stan_fit.stan_variable('nu_2')[:,0],
        stan_fit.stan_variable('nu_2')[:,1],
        stan_fit.stan_variable('tau'),
        ]).transpose()

    forward = np.quantile(
        np.array([bf_forward, stan_fit.stan_variable('filtering')]), 
        q=[0.005, 0.5, 0.995], axis=1
    )

    backward = np.quantile(
        np.array([bf_backward, stan_fit.stan_variable('backward_filtering')]), 
        q=[0.005, 0.5, 0.995], axis=1
    )

    smoothing = np.quantile(
        np.array([bf_smoothing, stan_fit.stan_variable('smoothing')]), 
        q=[0.005, 0.5, 0.995], axis=1
    )

    subject_folder = os.path.join('dutilh-results', subject)
    os.makedirs(subject_folder, exist_ok=True)

    fig, _ = plot_marginal_samples(Prior=prior, BayesFlow=bf_posterior, Stan=stan_posterior,
                                   param_names=model.param_names['constrained'],
                                   colors=colors)
    plt.savefig(os.path.join(subject_folder, 'marginal-prior-posterior.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_marginal_samples(BayesFlow=bf_posterior, Stan=stan_posterior,
                                   param_names=model.param_names['constrained'],
                                   colors=colors)
    
    plt.savefig(os.path.join(subject_folder, 'marginal-posterior.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_joint_samples(Prior=prior, BayesFlow=bf_posterior, Stan=stan_posterior, 
                                param_names=model.param_names['constrained'], 
                                colors=colors)
    plt.savefig(os.path.join(subject_folder, 'joint-prior-posterior.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_joint_samples(BayesFlow=bf_posterior, Stan=stan_posterior, 
                                param_names=model.param_names['constrained'], 
                                colors=colors)
    plt.savefig(os.path.join(subject_folder, 'joint-posterior.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_joint_samples(Prior=prior, BayesFlow=bf_posterior,
                                param_names=model.param_names['constrained'], 
                                colors=colors)
    plt.savefig(os.path.join(subject_folder, 'joint-prior-posterior-bf.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_joint_samples(Prior=prior, Stan=stan_posterior, 
                                param_names=model.param_names['constrained'], 
                                colors=colors)
    plt.savefig(os.path.join(subject_folder, 'joint-prior-posterior-stan.png'), bbox_inches='tight')
    plt.close()

    fig, _ = plot_classification(forward, 'Filtering')
    plt.savefig(os.path.join(subject_folder, 'classification-filtering.png'))
    plt.close()

    fig, _ = plot_classification(backward, 'Backward Filtering')
    plt.savefig(os.path.join(subject_folder, 'classification-backward-filtering.png'))
    plt.close()

    fig, _ = plot_classification(smoothing, 'Smoothing')
    plt.savefig(os.path.join(subject_folder, 'classification-smoothing.png'))
    plt.close()

    fig, _ = plot_data(rts, choices)
    plt.savefig(os.path.join(subject_folder, 'data.png'))
    plt.close()

    fig = plot_predictive_checks_histograms(bf_smoothing, bf_posterior, rts, choices)
    plt.savefig(os.path.join(subject_folder, 'posterior-predictives-smoothing-bf.png'))
    plt.close()

    fig = plot_predictive_checks_histograms(stan_fit.stan_variable('smoothing'), stan_posterior, rts, choices)
    plt.savefig(os.path.join(subject_folder, 'posterior-predictives-smoothing-stan.png'))
    plt.close()